<a href="https://colab.research.google.com/github/Manoj1joshi/hindi-lyrics-generator/blob/main/hindi_lyrics_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving kaggle.json to kaggle (1).json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d stutig29/hinglish-hindi-parallel-corpus
!unzip -q hinglish-hindi-parallel-corpus.zip


Dataset URL: https://www.kaggle.com/datasets/stutig29/hinglish-hindi-parallel-corpus
License(s): CC0-1.0
  0% 0.00/11.1k [00:00<?, ?B/s]
100% 11.1k/11.1k [00:00<00:00, 21.8MB/s]


In [ ]:
# Load Hindi lyrics
import numpy as np
with open("hindi_lyrics.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(f"Total characters: {len(text)}")

# Create vocabulary and mapping
vocab = sorted(set(text))
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = np.array(vocab)

def vectorize_string(string):
    return np.array([char2idx[c] for c in string], dtype=np.int32)

vectorized_text = vectorize_string(text)


Total characters: 16266


In [ ]:
!pip install gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


Now that the `gtts` library is installed, I will remove the redundant code from the selected cell and rerun it.

In [ ]:
# ✅ Hindi Lyrics LSTM Model using PyTorch - Train & Generate

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
from google.colab import files
from gtts import gTTS
from IPython.display import Audio

# --- 3. Character Mapping ---
# Use the text loaded from the previous cell
vocab = sorted(set(text))
char2idx = {ch: idx for idx, ch in enumerate(vocab)}
idx2char = {idx: ch for idx, ch in enumerate(vocab)}
text_as_int = [char2idx[c] for c in text]

# --- 4. Dataset Preparation ---
class HindiLyricsDataset(Dataset):
    def __init__(self, text_as_int, seq_length):
        self.seq_length = seq_length
        self.data = text_as_int

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        return (
            torch.tensor(self.data[idx:idx+self.seq_length]),
            torch.tensor(self.data[idx+1:idx+self.seq_length+1])
        )

seq_length = 100
batch_size = 64
dataset = HindiLyricsDataset(text_as_int, seq_length)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# --- 5. Define Model ---
class CharLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharLSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        if hidden is None:
            out, hidden = self.lstm(x, hidden)
        else:
            out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

vocab_size = len(vocab)
model = CharLSTM(vocab_size, embed_dim=256, hidden_dim=512).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

# --- 6. Training ---
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EPOCHS = 10
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (x, y) in enumerate(data_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output, _ = model(x)
        loss = criterion(output.view(-1, vocab_size), y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss / len(data_loader):.4f}")

# --- 7. Text Generation ---
def generate_text(model, start_string, char2idx, idx2char, generation_length=500, temperature=1.0):
    model.eval()
    input_eval = torch.tensor([char2idx[c] for c in start_string], dtype=torch.long).unsqueeze(0).to(device)
    hidden = None
    generated = start_string

    for _ in range(generation_length):
        output, hidden = model(input_eval, hidden)
        output = output[:, -1, :] / temperature
        probabilities = F.softmax(output, dim=-1).squeeze()
        next_char_idx = torch.multinomial(probabilities, 1).item()
        generated += idx2char[next_char_idx]
        input_eval = torch.tensor([[next_char_idx]], dtype=torch.long).to(device)

    return generated

print("\n\n🎵 Generated Hindi Lyrics 🎵\n")
lyrics = generate_text(model, start_string="तू ", char2idx=char2idx, idx2char=idx2char)
print(lyrics)

# --- 8. Convert to Audio ---
tts = gTTS(text=lyrics, lang='hi')
tts.save("generated_song.mp3")
Audio("generated_song.mp3")

Epoch 1/10, Loss: 0.7107
Epoch 2/10, Loss: 0.1030
Epoch 3/10, Loss: 0.0937
Epoch 4/10, Loss: 0.0900
Epoch 5/10, Loss: 0.0878
Epoch 6/10, Loss: 0.0864
Epoch 7/10, Loss: 0.0850
Epoch 8/10, Loss: 0.0842
Epoch 9/10, Loss: 0.0830
Epoch 10/10, Loss: 0.0827


🎵 Generated Hindi Lyrics 🎵

तू 
जब तक जहान में सुबह शाम है
तब तक मेरे नाम तू
जब तक जहान में मेरा नाम है
तब तक मेरे नाम तूउलझन भी हूँ तेरी
उलझन का हल भी हूँ मैं
थोड़ा सा जिद्दी हूँ
थोड़ा पागल भी हूँ मैंबरखा बिजली बादल झूठे
झूठी फूलों की सौगातें
सच्ची तू है सच्चा मैं हूँ

सच्ची अपने दिल की बातें 
दस्तख़त हाथों से हाथों पे कर दे तू
ना कर आँखों पे पलकों के परदे तू
क्या ये इतना बड़ा काम है ऐलान हैजब तक जहान में सुबह शाम है
तब तक मेरे नाम तू
जब तक जहान में मेरा नाम है
तब तक मेरे नाम तूमेरे ही घेरे में घूमेगी हर पल तू ऐसे
सूरज के घे


In [ ]:
print("\n\n🎵 Generated Hindi Lyrics 🎵\n")
# Change the prompt here to anything meaningful
lyrics = generate_text(model, start_string="चांदनी ", char2idx=char2idx, idx2char=idx2char)
print(lyrics)

# --- 8. Convert to Audio ---
tts = gTTS(text=lyrics, lang='hi')
tts.save("generated_song.mp3")
Audio("generated_song.mp3")




🎵 Generated Hindi Lyrics 🎵

चांदनी में मारूँ तो छोरी पट जाए
छोरी पट जाएभंकसदिल में तेरे क्या है मुझको बता दे
तेरे मेरे बीच का ये पर्दा हटा देआजा मेरी रानी तुझे गले से लगाऊं
ब्याकुल जिया मेरा बस यही चाहेंतेरे पास तो
जो तेरे प्यार कितना   
# बारिश 
# हमने रेत पे   
# गहरा हल्का  
# हश्र से पहले   तो क्या हुआ जुदा हुए
मगर है खुशी मिले तो थे
तो क्या हुआ मुड़े रास्ते
कुछ दूर संग चले तो थेदोबारा मिलेंगे किसी मोड़ पे
जो बाकी है वो बात होगी कभी
चलो आज चलते हैं हमफिर मुलाकात होगी कभी
फिर मुलाकात होगी कभी
जुदा हो रहे हैं कदम
फिर मुलाकात होग
